Load the data.

In [2]:
# Code adapted from Harish Satgunarajah
import pandas as pd

df = pd.read_csv("FIBAData.csv", index_col = 0, encoding='latin-1')
df.head()

,Region,Date,Away Team,Home Team,Result,Team,Period,Clock,Player,Play Details,Away Score,Home Score
ID,,,,,,,,,,,,
1,2019 World Cup,9/15/2019,Spain,Argentina,Miss 2 Pts,Spain,1,9:39,Ricky Rubio,9 Ricky Rubio > Hand Off > From Stationary > R...,0,0
2,2019 World Cup,9/15/2019,Spain,Argentina,NaN,NaN,1,9:39,NaN,Shot > Ricky Rubio > Any Type > 2 Point Attemp...,0,0
3,2019 World Cup,9/15/2019,Spain,Argentina,NaN,NaN,1,9:39,NaN,Misc Stat > Defensive Rebound > Marcos Delia,0,0
4,2019 World Cup,9/15/2019,Spain,Argentina,Miss 3 Pts,Argentina,1,9:30,Luis Scola,7 Facundo Campazzo > P&R Ball Handler > High P...,0,0
5,2019 World Cup,9/15/2019,Spain,Argentina,NaN,NaN,1,9:30,NaN,Shot > Luis Scola > Any Type > 3 Point Attempt...,0,0


Let's add a column with an ID for each unique game.

In [3]:
df['GameID'] = df['Region'] + str(df['Date']) + df['Home Team'] + df['Away Team']
games = df['GameID'].unique().tolist()
df['GameID'] = df['GameID'].apply(lambda x: games.index(x))

We can see which teams we have the most/least games for.

In [4]:
HTs = df.groupby(['Home Team'])['GameID'].nunique()
ATs = df.groupby(['Away Team'])['GameID'].nunique()
HTs.add(ATs, fill_value=0).astype(int).sort_values(ascending=False)

Serbia          65
France          64
Spain           63
Lithuania       58
Argentina       53
                ..
Saudi Arabia     3
South Africa     3
Kuwait           2
Singapore        2
Bahrain          1
Name: GameID, Length: 93, dtype: int64

In [4]:
teams = set(df['Home Team'].unique()).union(set(df['Away Team'].unique()))
len(teams)

93

Let's see how many players we have data on

In [5]:
df['Player'].value_counts()

Luis Scola            1286
Bojan Bogdanovic      1151
Bogdan Bogdanovic     1023
Facundo Campazzo       905
Andray Blatche         830
                      ... 
Mario Lopez              1
Nadim Soufiane           1
Satkeyev Aimurat         1
Wenbo Lu                 1
Doumgoto Djikoloum       1
Name: Player, Length: 2873, dtype: int64

We seem to have 2873, but I wonder if there are any duplicates in here with slight differences in spelling (e.g. capitalization, accents, middle names). However, cleaning this seems difficult...

In [6]:
import unidecode
player_names = set(df['Player'].unique())
decoded_player_names = set(df['Player'].apply(lambda x: unidecode.unidecode(str(x))).unique())
capital_player_names = set(df['Player'].str.upper().unique())
capital_decoded_player_names = set(df['Player'].apply(lambda x: unidecode.unidecode(str(x).upper())).unique())
print(len(player_names), len(decoded_player_names))
print(len(capital_player_names), len(capital_decoded_player_names))
print(player_names - decoded_player_names)
print(decoded_player_names - player_names)

2874 2870
2860 2856
{nan, 'Tomás Satoransky', 'Jakub Kudlácek', 'Sergio Rodríguez Gómez', 'Leonardo Gutiérrez', 'Martin Kríz', 'Ondrej Balvín', 'Andrés Nocioni', 'Sergio Rodríguez', 'Tomás Vyoral', 'José Calderon', 'Martin KrÃ\xadz', 'José-Manuel Calderon', 'Fidel Mendoça', 'Ondrej BalvÃ\xadn', 'João Paulo Batista', 'Hidayet Türkoglu'}
{'Martin KrAz', 'Leonardo Gutierrez', 'Andres Nocioni', 'Jakub Kudlacek', 'Martin Kriz', 'Jose-Manuel Calderon', 'nan', 'Hidayet Turkoglu', 'Fidel Mendoca', 'Jose Calderon', 'Sergio Rodriguez Gomez', 'Ondrej BalvAn', 'Sergio Rodriguez'}


In [7]:
import difflib #https://docs.python.org/3/library/difflib.html
players = df[df['Team'] == 'Spain']['Player'].unique().tolist()
difflib.get_close_matches('Jose Calderon', players)

['José Calderon', 'José-Manuel Calderon']

Let's see whether we need to clean the period data.

In [8]:
period_list = df.groupby('GameID')['Period'].unique()
print("Periods in data  :", df['Period'].unique().tolist())
print("Number of Games  :", len(period_list))
for i in range(1, 5):
    print("Missing period", i, ":", period_list.apply(lambda x: i not in x).sum())
for (i, j) in ((5, 6), (5, 7), (6, 7)):
    print("Has", j, "but not", i, " :", period_list.apply(lambda x: j in x and i not in x).sum())

Periods in data  : [1, 2, 3, 4, 5, 6, 7]
Number of Games  : 1164
Missing period 1 : 10
Missing period 2 : 33
Missing period 3 : 20
Missing period 4 : 25
Has 6 but not 5  : 12
Has 7 but not 5  : 0
Has 7 but not 6  : 6


We may wish to disregard the games with these exceptions as being unreliable, or we can ignore these errors. Let us also see how many plays we have per period.

In [9]:
df.groupby(['GameID', 'Period']).size().describe()

count    4684.000000
mean      168.519641
std        37.370304
min         6.000000
25%       149.000000
50%       166.000000
75%       183.000000
max       702.000000
dtype: float64

As we see, some periods include only 6 rows of information, while some include up to 702. This is further evidence that the period labels may not be reliable in some cases. In the future, we could analyze the clock column to ensure validity of the period labels.

Below are some ideas for parsing the plays, if we were to do this from scratch.

In [10]:
success_type = {"Make 2 Pts" : 2, "Make 3 Pts" : 3, "1 Pts" : 1, "Made" : 1, "Foul": 0.1}

def play_distribution(player, gameID):
    """
    Return player's play type distribution for given game.
    Synergy actions are PnR Handler, Spot Up, Transition, Isolation, Post Up, PnR Man, Cut, ORB, Off Scree, Hand Off
    player: String, player's full name
    gameID: int, game ID from df
    return: dict, {play_type: [attempts, pts]}
    """
    # TODO Handle non-plays? (free throws, turnovers, fouls, etc.)
    # What result for foul? Count number of shots received? Right now adding 0.1...
    # If assisted by different player, count under resulting play type
    play_distribution = {}
    temp = df[df['GameID'] == gameID]
    temp = temp[temp['Play Details'].str.contains(player)]['Play Details']
    for play in temp.tolist():
        play = play.split(' > ')
        if player in play[0] or 'Ball Delivered' in play:
            play_type = play[1 if player in play[0] else play.index('Ball Delivered') + 2]
            play_result = play[-1]
            if play_type in play_distribution:
                play_distribution[play_type][0] += 1
                play_distribution[play_type][1] += success_type[play_result] if play_result in success_type else 0
            else:
                play_distribution[play_type] = [1, success_type[play_result] if play_result in success_type else 0]
    return play_distribution

play_distribution("Ricky Rubio", 0)

{'Hand Off': [3, 2],
 'P&R Ball Handler': [11, 4.199999999999999],
 'Non Possession': [8, 0],
 'Offensive Rebound': [2, 2],
 'Spot-Up': [3, 5],
 'Cut': [1, 1],
 'No Play Type': [3, 0.1],
 'Transition': [1, 0]}

In [34]:
def play_counts(player, gameID):
    """
    Return player's play type counts for game with given gameID.
    Synergy actions: PnR Handler, Spot Up, Transition, Isolation, Post Up, PnR Man, Cut, ORB, Off Screen, Hand Off
    player: String, player's full name as appears in data
    gameID: int, game ID from df
    return: dict, {play_type: count}
    """
    play_distribution = {}
    temp = df[df['GameID'] == gameID]
    temp = temp[temp['Play Details'].str.contains(player)]['Play Details']
    for play in temp.tolist():
        play = play.split(' > ')
        if player in play[0] or 'Ball Delivered' in play: # If assisted by different player, count under resulting play type
            play_type = play[1 if player in play[0] else play.index('Ball Delivered') + 2]
            if play_type in play_distribution:
                play_distribution[play_type] += 1
            elif play_type != 'Non Possession' and play_type != 'No Play Type':
                play_distribution[play_type] = 1
    return play_distribution

play_counts("Ricky Rubio", 0)

{'Hand Off': 3,
 'P&R Ball Handler': 11,
 'Offensive Rebound': 2,
 'Spot-Up': 3,
 'Cut': 1,
 'Transition': 1}

In [12]:
unique = set()
temp = df['Play Details']
for play in temp.tolist():
    play = play.split(' > ')
    if len(play) >= 2 and play[1] == 'Non Possession':
        unique.add(play[2])
unique

{'8 Sec Violation',
 'Free Throw',
 'Kicked Ball',
 'Non Shooting Foul',
 'Out of Bound 5 Sec Violation',
 'Shot Clock Violation'}

In [13]:
unique = set()
temp = df['Play Details']
for play in temp.tolist():
    play = play.split(' > ')
    if play[0] == 'Misc Stat':
        unique.add(play[1])
unique

{'Assist',
 'Block',
 'Defensive Rebound',
 'Offensive Rebound',
 'Steal',
 'Team Defensive Rebound',
 'Team Offensive Rebound'}

In [14]:
unique = set()
temp = df['Play Details']
for play in temp.tolist():
    play = play.split(' > ')
    if play[0] == 'Shot':
        unique.add(play[-1])
unique

{'Made', 'Make 2 Pts', 'Make 3 Pts', 'Miss 2 Pts', 'Miss 3 Pts', 'Missed'}

In [15]:
unique = set()
temp = df['Play Details']
for play in temp.tolist():
    play = play.split(' > ')
    if play[0].isalpha():
        unique.add(play[0])
unique

{'Shot', 'Turnover'}

In [16]:
unique = set()
temp = df['Play Details']
for play in temp.tolist():
    play = play.split(' > ')
    if not any(map(str.isdigit, play[0])):
        unique.add(play[0])
unique

{'Misc Stat',
 'Personal Foul',
 'Player Movement',
 'Shot',
 'Technical Foul',
 'Time Event',
 'Turnover'}

In [14]:
player_names = set(df['Player'].dropna().unique())
player_names

{'Titus Nicoara',
 'Beau Belga',
 'John Hernandez',
 'Ingus Jakovics',
 'Bruno Fitipaldo',
 'Nathan Jawai',
 'Joaquin Rodriguez',
 'Kevin Durant',
 'Eric Moreland',
 'Jean-Francois Kebe',
 'Mohammed Al Marwani',
 'Maarten Bouwknecht',
 'Shuo Fang',
 'Tomas Kyzlink',
 'Joel Bolomboy',
 'Fadel Alnajjar',
 'Clevin Hannah',
 'Kevin Jones',
 'Leonardo Gutiérrez',
 'Marko Popovic',
 'Siu Wing Chan',
 'Andrejs Grazulis',
 'Maik Kotsar',
 'Mahmoud Abdeen',
 'Ziyed Chennoufi',
 'Morlaye Bangoura',
 'Janari Joesaar',
 'Anderson Varejao',
 'Evariste Shonganya',
 'Aleksandar Kostoski',
 'Jhonatan Luz dos Santos',
 'Walter Hodge',
 'Jerome Jordan',
 'Jun-beom Jeon',
 'Thomas Abercrombie',
 'Chin Yong Wong',
 'Jaraun Burrows',
 'Daniel Johnson',
 'Sasu Salin',
 'Elie Chamoun',
 'Rienk Mast',
 'Sviatoslav Mikhailiuk',
 'Alex Abrines',
 'Julian Williams-Washburn',
 'Abdoulaye Ndoye',
 'Orlando Mendez',
 'Kevin Rubio',
 'Sertac Sanli',
 'Vlado Ilievski',
 'Akaki Dvalishvili',
 'Weyinmi Rose',
 'Shiu Wa

In [42]:
player_counts = {}
for p in player_names:
    if p not in player_counts:
        player_counts[p] = {}
player_counts

{'Titus Nicoara': {},
 'Beau Belga': {},
 'John Hernandez': {},
 'Ingus Jakovics': {},
 'Bruno Fitipaldo': {},
 'Nathan Jawai': {},
 'Joaquin Rodriguez': {},
 'Kevin Durant': {},
 'Eric Moreland': {},
 'Jean-Francois Kebe': {},
 'Mohammed Al Marwani': {},
 'Maarten Bouwknecht': {},
 'Shuo Fang': {},
 'Tomas Kyzlink': {},
 'Joel Bolomboy': {},
 'Fadel Alnajjar': {},
 'Clevin Hannah': {},
 'Kevin Jones': {},
 'Leonardo Gutiérrez': {},
 'Marko Popovic': {},
 'Siu Wing Chan': {},
 'Andrejs Grazulis': {},
 'Maik Kotsar': {},
 'Mahmoud Abdeen': {},
 'Ziyed Chennoufi': {},
 'Morlaye Bangoura': {},
 'Janari Joesaar': {},
 'Anderson Varejao': {},
 'Evariste Shonganya': {},
 'Aleksandar Kostoski': {},
 'Jhonatan Luz dos Santos': {},
 'Walter Hodge': {},
 'Jerome Jordan': {},
 'Jun-beom Jeon': {},
 'Thomas Abercrombie': {},
 'Chin Yong Wong': {},
 'Jaraun Burrows': {},
 'Daniel Johnson': {},
 'Sasu Salin': {},
 'Elie Chamoun': {},
 'Rienk Mast': {},
 'Sviatoslav Mikhailiuk': {},
 'Alex Abrines': 

In [44]:
# is_rubio = df['Player'] == 'Ricky Rubio'
# rubio = df[is_rubio]
# #games = rubio['GameID']
# games = set(rubio['GameID'].unique())
# for g in games:
#     player_counts['Ricky Rubio'][g] = play_counts('Ricky Rubio', g)
#for each player collect the GameIDs of the games that they played
for p in player_names:
    is_player = df['Player'] == p
    player = df[is_player]
    games = set(player['GameID'].unique())
    for g in games:
        player_counts[p][g] = play_counts(p, g)


In [47]:
player_counts['Ricky Rubio']

{0: {'Hand Off': 3,
  'P&R Ball Handler': 11,
  'Offensive Rebound': 2,
  'Spot-Up': 3,
  'Cut': 1,
  'Transition': 1},
 5: {'P&R Ball Handler': 21,
  'Spot-Up': 5,
  'Transition': 1,
  'Hand Off': 2,
  'ISO': 2},
 10: {'P&R Ball Handler': 19,
  'Spot-Up': 4,
  'Off Screen': 1,
  'Offensive Rebound': 1,
  'ISO': 1},
 19: {'Spot-Up': 9,
  'P&R Ball Handler': 9,
  'Cut': 1,
  'Hand Off': 2,
  'Offensive Rebound': 2,
  'ISO': 1},
 1046: {'Spot-Up': 3,
  'P&R Ball Handler': 15,
  'Off Screen': 1,
  'Transition': 1},
 1049: {'Cut': 1,
  'Hand Off': 1,
  'Transition': 1,
  'P&R Ball Handler': 3,
  'Off Screen': 1,
  'Spot-Up': 1},
 670: {'P&R Ball Handler': 8, 'Spot-Up': 3},
 672: {'P&R Ball Handler': 5, 'Transition': 1, 'Offensive Rebound': 1},
 1056: {'Transition': 1,
  'P&R Ball Handler': 6,
  'Cut': 1,
  'Offensive Rebound': 1,
  'Spot-Up': 1},
 1058: {'P&R Ball Handler': 4, 'Transition': 1, 'Spot-Up': 1},
 35: {'P&R Ball Handler': 11,
  'Off Screen': 1,
  'ISO': 1,
  'Spot-Up': 2,
  'Tr